In [9]:
from ..utils.feature_selection import get_X_y
import pandas as pd
import pyprojroot

ImportError: attempted relative import with no known parent package

In [ ]:
DATA_PATH = pyprojroot.here().joinpath('data', 'fetal_health.csv')
X, y = get_X_y(data_path=DATA_PATH, target_variable='fetal_health')